In [9]:
import bs4
import json
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [7]:
driver = webdriver.Chrome()

url = 'https://cafe.naver.com/mbticafe'
driver.get(url)

driver.switch_to.frame("cafe_main")

text = driver.find_element(By.XPATH, '//*[@id="SE-B7B6F625-6A27-4E8B-9A34-58AE61067AB6"]').text

print(text)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="SE-B7B6F625-6A27-4E8B-9A34-58AE61067AB6"]"}
  (Session info: chrome=118.0.5993.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF778BF8EF2+54786]
	(No symbol) [0x00007FF778B65612]
	(No symbol) [0x00007FF778A1A64B]
	(No symbol) [0x00007FF778A5B79C]
	(No symbol) [0x00007FF778A5B91C]
	(No symbol) [0x00007FF778A96D87]
	(No symbol) [0x00007FF778A7BEAF]
	(No symbol) [0x00007FF778A94D02]
	(No symbol) [0x00007FF778A7BC43]
	(No symbol) [0x00007FF778A50941]
	(No symbol) [0x00007FF778A51B84]
	GetHandleVerifier [0x00007FF778F47F52+3524194]
	GetHandleVerifier [0x00007FF778F9D800+3874576]
	GetHandleVerifier [0x00007FF778F95D7F+3843215]
	GetHandleVerifier [0x00007FF778C95086+694166]
	(No symbol) [0x00007FF778B70A88]
	(No symbol) [0x00007FF778B6CA94]

In [39]:
driver = webdriver.Chrome()

url = 'https://cafe.naver.com/mbticafe'
driver.get(url)

driver.find_element(By.ID, "menuLink18").click()

# driver.switch_to.frame("cafe_main")

main_content = []
for i in range(1, 16):
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.XPATH, f'//*[@id="main-area"]/div[4]/table/tbody/tr[{i}]/td[1]/div[2]/div/a').click()
    time.sleep(1)  
    text = driver.find_elements(By.CSS_SELECTOR, '.se-fs-.se-ff-')
    for j in text:
        main_content.append(j.text)
    driver.back()

main_content

['enfp esfp 나오는 제 남친 특징인데요 원래 엔프피 남자 이러는지 궁금해서요',
 '',
 '1. 비행기, 열차 등 오랜시간 기다리고 한곳에 머물기 싫어하고 시간 아까워 함 (잠자는 것도 시간 아까워함) 그래서 여행가는 것 싫어하고 빠른 환승 칸 찾는 거 필수',
 '',
 '2. 그렇다고 생산적인 걸 하는 성격은 아니고 게임하거나 친구들이랑 놀거나 강아지랑 산책하는 것 좋아함',
 '',
 '3. 굉장히 정의롭고 예의 중시하고 남한테 피해주는 것 싫어함',
 '',
 '4. 낯가림 있고 이성이랑 말 잘 못하고 말할 생각도 없음',
 '',
 '5. 남자라 그런가 말투나 가치관이 t성향이 깔려있는데 상처 잘 받음 근데 금방 까먹음',
 '',
 '6. 미래, 망상 얘기 좋아하고 그런 질문 많이 하는데 정작 자신은 그런거에 대답 대충함 걍 물어보고 사람 곤란하게 만들기를 좋아함',
 '',
 '7. 일하기 싫어하는데 막상 시키는 건 잘함',
 '',
 '8. 귀찮다는 말 달고 사는데 활동량은 많음',
 '',
 '9. 호기심은 많은데 막상 실행하고 싶어하는 건 까다로워서 별로 없음',
 '',
 '10. 걸어다니면서 노래부르고 정신산만한 편',
 '',
 '요정도인데 생각보다 내향적이기도 하네요 근데 i 같진 않아요 절대 엔프피분들 공감하시는지!!',
 '전화와서 한시간씩 스몰토크 하는데 호감이 맞겠죠…?',
 '직장생활 하다가',
 '프리 전향한지 3개월째입니다',
 '혹시 프리이신분들 계실까요?',
 '저는 프리랜서 강사쪽이고 분야는 디지털기술인데',
 '어떤 일하는지, 만족도는 어떤지 공유해보아요~',
 '제가 어제 호감있는 사내 여자 엔프피분에게 술을 먹자고 해서 먹었습니다 여자 엔프피분이 만나는 사람이 있냐는 질문을 했는데 호감이 있는 질문일까요??',
 '인프피남자 귀여워죽겠어요 1차원적이아니라 하는행동이나 생각이 너무 귀여운가같아요 ㅋㅋㅋㅋ',
 '생각했던 이미지는 소심하고 얌전하고 소극적이고 조금은 지루할수도있다고 생각했는데 한번 데이트해

In [40]:
a = main_content.copy()

In [41]:
a

['enfp esfp 나오는 제 남친 특징인데요 원래 엔프피 남자 이러는지 궁금해서요',
 '',
 '1. 비행기, 열차 등 오랜시간 기다리고 한곳에 머물기 싫어하고 시간 아까워 함 (잠자는 것도 시간 아까워함) 그래서 여행가는 것 싫어하고 빠른 환승 칸 찾는 거 필수',
 '',
 '2. 그렇다고 생산적인 걸 하는 성격은 아니고 게임하거나 친구들이랑 놀거나 강아지랑 산책하는 것 좋아함',
 '',
 '3. 굉장히 정의롭고 예의 중시하고 남한테 피해주는 것 싫어함',
 '',
 '4. 낯가림 있고 이성이랑 말 잘 못하고 말할 생각도 없음',
 '',
 '5. 남자라 그런가 말투나 가치관이 t성향이 깔려있는데 상처 잘 받음 근데 금방 까먹음',
 '',
 '6. 미래, 망상 얘기 좋아하고 그런 질문 많이 하는데 정작 자신은 그런거에 대답 대충함 걍 물어보고 사람 곤란하게 만들기를 좋아함',
 '',
 '7. 일하기 싫어하는데 막상 시키는 건 잘함',
 '',
 '8. 귀찮다는 말 달고 사는데 활동량은 많음',
 '',
 '9. 호기심은 많은데 막상 실행하고 싶어하는 건 까다로워서 별로 없음',
 '',
 '10. 걸어다니면서 노래부르고 정신산만한 편',
 '',
 '요정도인데 생각보다 내향적이기도 하네요 근데 i 같진 않아요 절대 엔프피분들 공감하시는지!!',
 '전화와서 한시간씩 스몰토크 하는데 호감이 맞겠죠…?',
 '직장생활 하다가',
 '프리 전향한지 3개월째입니다',
 '혹시 프리이신분들 계실까요?',
 '저는 프리랜서 강사쪽이고 분야는 디지털기술인데',
 '어떤 일하는지, 만족도는 어떤지 공유해보아요~',
 '제가 어제 호감있는 사내 여자 엔프피분에게 술을 먹자고 해서 먹었습니다 여자 엔프피분이 만나는 사람이 있냐는 질문을 했는데 호감이 있는 질문일까요??',
 '인프피남자 귀여워죽겠어요 1차원적이아니라 하는행동이나 생각이 너무 귀여운가같아요 ㅋㅋㅋㅋ',
 '생각했던 이미지는 소심하고 얌전하고 소극적이고 조금은 지루할수도있다고 생각했는데 한번 데이트해

In [51]:
filtered_list = [x for x in a if x != '']

In [52]:
filtered_list

['enfp esfp 나오는 제 남친 특징인데요 원래 엔프피 남자 이러는지 궁금해서요',
 '1. 비행기, 열차 등 오랜시간 기다리고 한곳에 머물기 싫어하고 시간 아까워 함 (잠자는 것도 시간 아까워함) 그래서 여행가는 것 싫어하고 빠른 환승 칸 찾는 거 필수',
 '2. 그렇다고 생산적인 걸 하는 성격은 아니고 게임하거나 친구들이랑 놀거나 강아지랑 산책하는 것 좋아함',
 '3. 굉장히 정의롭고 예의 중시하고 남한테 피해주는 것 싫어함',
 '4. 낯가림 있고 이성이랑 말 잘 못하고 말할 생각도 없음',
 '5. 남자라 그런가 말투나 가치관이 t성향이 깔려있는데 상처 잘 받음 근데 금방 까먹음',
 '6. 미래, 망상 얘기 좋아하고 그런 질문 많이 하는데 정작 자신은 그런거에 대답 대충함 걍 물어보고 사람 곤란하게 만들기를 좋아함',
 '7. 일하기 싫어하는데 막상 시키는 건 잘함',
 '8. 귀찮다는 말 달고 사는데 활동량은 많음',
 '9. 호기심은 많은데 막상 실행하고 싶어하는 건 까다로워서 별로 없음',
 '10. 걸어다니면서 노래부르고 정신산만한 편',
 '요정도인데 생각보다 내향적이기도 하네요 근데 i 같진 않아요 절대 엔프피분들 공감하시는지!!',
 '전화와서 한시간씩 스몰토크 하는데 호감이 맞겠죠…?',
 '직장생활 하다가',
 '프리 전향한지 3개월째입니다',
 '혹시 프리이신분들 계실까요?',
 '저는 프리랜서 강사쪽이고 분야는 디지털기술인데',
 '어떤 일하는지, 만족도는 어떤지 공유해보아요~',
 '제가 어제 호감있는 사내 여자 엔프피분에게 술을 먹자고 해서 먹었습니다 여자 엔프피분이 만나는 사람이 있냐는 질문을 했는데 호감이 있는 질문일까요??',
 '인프피남자 귀여워죽겠어요 1차원적이아니라 하는행동이나 생각이 너무 귀여운가같아요 ㅋㅋㅋㅋ',
 '생각했던 이미지는 소심하고 얌전하고 소극적이고 조금은 지루할수도있다고 생각했는데 한번 데이트해보니까 더좋아젔어요 의외로 걱정했던 부분은 안그랬고',
 '장점으로 생각했던 다정함 섬세함 이런게 

In [54]:


with open('my_list.txt', 'w', encoding='utf-8') as file:
    for item in filtered_list:
        file.write(str(item) + '\n')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
url = 'https://cafe.naver.com/mbticafe'
driver.get(url)
driver.find_element(By.ID, "menuLink18").click()
driver.switch_to.frame("cafe_main")




for post_num in range(1, 16):
    post_xpath = f'//*[@id="main-area"]/div[4]/table/tbody/tr[{post_num}]/td[1]/div[2]/div/a'
    driver.find_element(By.XPATH, post_xpath).click()
    time.sleep(1)  

    comments = driver.find_elements(By.XPATH, '//span[@class="text_comment"]')

    print(f"게시글 {post_num})의 댓글:")
    for comment in comments:
        comment_text = comment.text

    
    driver.execute_script("window.history.go(-1)")
    time.sleep(1)  


driver.quit()

In [3]:
# 크롤링 과정
import time


driver = webdriver.Chrome() 

# 경로 설정
url = 'https://cafe.naver.com/mbticafe'

# a = driver.find_element(By.ID, "menuLink18")      # INFP ENFP로 접속
# a.click()

# 클럽 아이디, 원하는 메뉴 아이디
club_id = 11856775
menu_id = 18 

# iframe 설정

# 에러시 딜레이
delay = 3

#카페 열기
driver.get(f'{url}?iframe_url=/ArticleList.nhn?search.clubid={club_id}%26search.menuid={menu_id}%26')
time.sleep(2)
driver.switch_to.frame("cafe_main")

#카테고리 첫 번째 글 클릭
driver.find_element(By.XPATH,'//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a').click()